In [40]:
import pandas as pd

# CSV 파일 읽기
df_solar = pd.read_csv('df_solar_final_sorted.csv')
df_all = pd.read_csv('df_all5.csv')

In [27]:
df_all.head()

,발전구분,호기,일자,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,총합,총발전량(kWh),연월,일수
0,감우리,1.0,2021-08-05,0.0,0.0,0.0,0.0,0.0,0.48,14.88,85.92,205.92,321.60,122.40,276.96,279.36,412.32,373.44,342.72,175.68,60.96,31.20,2.88,0.0,0.0,0.0,0.0,2706.72,2706.72,2021-08,31
1,감우리,1.0,2021-08-06,0.0,0.0,0.0,0.0,0.0,0.48,21.60,91.20,209.76,320.16,363.84,364.80,444.00,207.36,173.76,144.48,164.64,33.60,21.60,0.48,0.0,0.0,0.0,0.0,2561.76,2561.76,2021-08,31
2,감우리,1.0,2021-08-07,0.0,0.0,0.0,0.0,0.0,0.00,12.48,90.72,214.56,234.24,353.28,380.64,418.56,390.72,398.40,152.64,124.32,108.48,27.36,0.96,0.0,0.0,0.0,0.0,2907.36,2907.36,2021-08,31
3,감우리,1.0,2021-08-08,0.0,0.0,0.0,0.0,0.0,0.48,16.80,73.44,163.68,249.60,184.32,259.20,157.44,153.12,97.44,38.40,34.08,24.00,8.64,0.48,0.0,0.0,0.0,0.0,1461.12,1461.12,2021-08,31
4,감우리,1.0,2021-08-09,0.0,0.0,0.0,0.0,0.0,0.00,7.20,96.00,206.88,143.04,237.12,371.04,442.08,347.52,323.52,358.08,257.28,129.12,34.56,0.96,0.0,0.0,0.0,0.0,2954.40,2954.40,2021-08,31


In [41]:
df_all.loc[
    (df_all['발전구분'] == '신인천해수구취수구') & (df_all['호기'].isna()),
    '호기'
] = '1.0'


C:\Users\박경국\AppData\Local\Temp\ipykernel_16596\1507354685.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_all.loc[


In [42]:
import pandas as pd

# ✅ 예시 데이터프레임: df_all
# 일자별 1~24시 시간별 발전량이 들어있는 wide format 형태
# ['발전구분', '호기', '일자', '1', '2', ..., '24', '총합', '총발전량(kWh)', '연월', '일수'] 등의 컬럼이 있음

# ✅ 시간 컬럼 리스트 생성 (문자열 '1' ~ '24')
hour_cols = [str(i) for i in range(1, 25)]

# ✅ long format으로 melt
df_long = df_all.melt(
    id_vars=['발전구분', '호기', '일자'],  # 고정 컬럼
    value_vars=hour_cols,              # 시간대 컬럼
    var_name='hour',                   # 시간대 변수 이름
    value_name='발전량(kWh)'           # 값 변수 이름
)

# ✅ 시간/일자 데이터 타입 처리
df_long['hour'] = df_long['hour'].astype(int)
df_long['일자'] = pd.to_datetime(df_long['일자'], errors='coerce')

# ✅ 날짜 분해 (year, month, day)
df_long['year'] = df_long['일자'].dt.year
df_long['month'] = df_long['일자'].dt.month
df_long['day'] = df_long['일자'].dt.day

# ✅ 요일 구하기 (0=월, 6=일)
df_long['weekday'] = df_long['일자'].dt.weekday

# ✅ 최종 컬럼 순서 재배열
df_long = df_long[['발전구분', '호기', 'year', 'month', 'day', 'hour', 'weekday', '발전량(kWh)']]

# ✅ 결과 확인
print(df_long.head())


  발전구분   호기  year  month  day  hour  weekday  발전량(kWh)
0  감우리  1.0  2021      8    5     1        3       0.0
1  감우리  1.0  2021      8    6     1        4       0.0
2  감우리  1.0  2021      8    7     1        5       0.0
3  감우리  1.0  2021      8    8     1        6       0.0
4  감우리  1.0  2021      8    9     1        0       0.0


In [44]:
import numpy as np

original_dict = {
    "감우리": {"1": 554},
    "남제주소내": {"1": 196},
    "무릉리": {"1": 809},
    "부산복합자재창고": {"1": 187},
    "부산본부": {"1": 392, "2": 502},
    "부산수처리장": {"1": 111},
    "부산신항": {"1": 115},
    "부산운동장": {"1": 1400},
    "삼척소내": {"1": 999, "2": 990, "3": 2002, "4": 2599},
    "송당리": {"1": 997},
    "신인천 북측부지": {"1": 365},
    "신인천소내": {"1": 200},
    "신인천전망대": {"1": 1742},
    "신인천해수구취수구": {"1": 907},
    "신인천 주차장": {"1": 603},
    "신풍리": {"1": 991},
    "영월본부": {"1": 40},
    "영월철도부지": {"1": 1191},
    "용수리": {"1": 889},
    "위미2리": {"1": 995},
    "이천D(백사면B)": {"1": 1000},
    "이천시 백사면A": {"1": 1834},
    "익산 다송리": {"1": 893},
    "인천수산정수장": {"1": 998},
    "하동공설운동장": {"1": 377},
    "하동변전소": {"1": 48},
    "하동보건소": {"1": 41},
    "하동본부": {"1": 997, "2": 1900, "3": 600, "4": 998, "5": 998, "6": 1000},
    "하동정수장": {"1": 93},
    "하동하수처리장": {"1": 54},
    "화촌주민참여형": {"1": 1000},
    "당진자재창고태양광": {"1": 700},
    "당진태양광": {"1": 1000},
    "당진화력수상태양광": {"1": 1000},
    "울산태양광#1": {"1": 500},
    "광양항광양냉장태양광": {"1": 700},
    "광양항어울림태양광": {"1": 600},
    "수원환경사업소태양광": {"1": 1500},
    "농촌공진도태양광": {"1": 994},
    "동해 바이오 연료저장고 지붕태양광": {"1": 1000},
}

# ✅ 호기 컬럼이 float형이면 str로 변환
df_long['호기'] = pd.to_numeric(df_long['호기'], errors='coerce')  # 안전하게 숫자 변환
df_long['호기'] = df_long['호기'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)


# ✅ MW 단위로 변환해서 매핑하는 함수
def get_capacity_mw(row):
    plant = row['발전구분']
    unit = row['호기']
    try:
        return original_dict[plant][unit] / 1000  # kW → MW 변환
    except KeyError:
        return np.nan

# ✅ 설비용량(MW) 컬럼 추가
df_long['설비용량(MW)'] = df_long.apply(get_capacity_mw, axis=1)

# ✅ 결과 확인 (중복 제거)
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.width', None)  # 줄 바꿈 없이 출력
pd.set_option('display.max_colwidth', None)  # 컬럼 너비 제한 해제

print(df_long[['발전구분', '호기', '설비용량(MW)']].drop_duplicates().sort_values(by=['발전구분', '호기']))



              발전구분 호기  설비용량(MW)
0              감우리  1     0.554
1304    광양항광양냉장태양광  1     0.700
2065     광양항어울림태양광  1     0.600
2976         남제주소내  1     0.196
7418      농촌공진도태양광  1     0.994
7783     당진자재창고태양광  1     0.700
8605         당진태양광  1     1.000
9877     당진화력수상태양광  1     1.000
10699          무릉리  1     0.809
12099     부산복합자재창고  1     0.187
16541         부산본부  2     0.502
18792       부산수처리장  1     0.111
21408         부산신항  1     0.115
25850        부산운동장  1     1.400
28466         삼척소내  1     0.999
31082         삼척소내  2     0.990
33569         삼척소내  3     2.002
35881         삼척소내  4     2.599
37920          송당리  1     0.997
39835   수원환경사업소태양광  1     1.500
40745     신인천 북측부지  1     0.365
42292      신인천 주차장  1     0.603
44185        신인천소내  1     0.200
48625       신인천전망대  1     1.742
51241    신인천해수구취수구  1     0.907
53495          신풍리  1     0.991
54895         영월본부  1     0.040
59336       영월철도부지  1     1.191
61862          용수리  1     0.889
63474      울산태양광#1  1     0.500
64751   

In [49]:
# CSV 파일로 저장 (UTF-8 인코딩, 인덱스 제외)
df_long.to_csv("df_all6.csv", index=False, encoding='utf-8-sig')